# Introduction to Random Sampling Strategies

In this section we will learn about different random sampling strategies and in which cases they are useful.

This notebook covers the following strategies:
- sampling from a normal distribution
- sampling from a uniform distribution
- sampling with a latin hypercube

You will explore the generated treatment combinations and compare them visually.

You are also going to scale the gathered treatments to a certain value range to keep the differently collected treatments as comparable as possible.

## Setup

First, we set the general parameters and methods which will be used at each stage of this notebook.

The parameters in this case are:
- the sample size
- the number of factors.
- the value range for the levels

In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
from typing import Iterable

np.random.seed(42)

n_samples = 100
n_factors = 2
condition_range = (0, 1)

### Scaling

Before gathering the different treatments, we will implement a scaling method.

This scaling method will scale the gathered treatments to a given range.

In [2]:
def scale(x, range: Iterable):
    """
    Scale the input to the given range.
    Args:
        x: the input to scale. Can be any type that can be converted to a numpy array.
        range: the range to scale to. Must be a tuple of length 2, with the first element being the minimum and the second element being the maximum.
    """
    assert len(range) == 2, "range must be an iterable of length 2."
    
    x = (x - np.min(x)) / (np.max(x) - np.min(x))
    x = x * (range[1] - range[0]) + range[0]

    return x

Does it work properly? Let's see!

In [ ]:
array = [1, 2, 3, 4, 5, 6]

array = scale(array, condition_range)

print(f"The scaled array is {array}.")

## Sampling from a normal distribution

The first sampling strategy which we will implement is sampling from a normal distribution.

We take the numpy.random.normal method.

You can take a look at the [official documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html), if you are not familiar with it.

Use the parameters from the Setup section.

In [4]:
# generate the treatment combinations with a normal distribution
# add your code here
treatments_normal = np.random.normal(size=(n_samples, n_factors))

# scale the treatment combinations to the range (-1, 1) with your scale function
# add your code here
treatments_normal = scale(treatments_normal, condition_range)

Did it work? Let's inspect the result in a scatter plot.

In [ ]:
plt.scatter(treatments_normal[:, 0], treatments_normal[:, 1])
plt.title('Random sampling from a normal distribution')
plt.xlabel('Factor 1')
plt.ylabel('Factor 2')
plt.show()

How does the result look like?

Which areas were covered well and which less?

Increase the sample size and re-run the code to see how the gathered treatmens change. 

## Sampling from a uniform distribution

The second sampling strategy which we will implement is sampling from a uniform distribution.

We take the numpy.random.normal method.

You can take a look at the [official documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.uniform.html), if you are not familiar with it.

Use the parameters from the Setup section.

In [6]:
treatments_uniform = None

# generate the treatment combinations with a uniform distribution
# add your code here
treatments_uniform = np.random.uniform(low=0, high=1, size=(n_samples, n_factors))

# We do not have to scale the uniform distribution in this case because it is already in the range (-1, 1)

Did it work? Let's inspect the result in a scatter plot.

In [ ]:
plt.scatter(treatments_uniform[:, 0], treatments_uniform[:, 1])
plt.title('Random sampling from a uniform distribution')
plt.xlabel('Factor 1')
plt.ylabel('Factor 2')
plt.show()

How does the result look like?

Which areas were covered well and which less?

Increase the sample size and re-run the code to see how the gathered treatmens change. 

## Sampling from a Latin Hypercube

Lastly, we will implement the latin hypercube strategy.

For that we will take the scipy.stats.qmc.LatinHypercube method.

Please make yourself familiar with it through the [official documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.LatinHypercube.html).

And here again - use the parameters from the setup.

Tip: We import the LatinHypercube class from scipy. Use the 'random' method of this class to create the treatment combinations.

In [8]:
from scipy.stats.qmc import LatinHypercube

# generate the treatment combinations with a normal distribution
# add your code here
latinhypercube = LatinHypercube(d=n_factors)
treatments_lhc = latinhypercube.random(n_samples)

# scale the treatment combinations to the range (-1, 1) with your scale function
# add your code here
treatments_lhc = scale(treatments_lhc, condition_range)

Did it work? Let's inspect the result in a scatter plot.

In [ ]:
plt.scatter(treatments_lhc[:, 0], treatments_lhc[:, 1])
plt.title('Random sampling from a Latin Hypercube')
plt.xlabel('Factor 1')
plt.ylabel('Factor 2')
plt.show()

How does the result look like?

Which areas were covered well and which less?

Does it look any different from the normal distribution?

Increase the sample size and re-run the code to see how the gathered treatmens change.

## Detailed analysis of the obtained results

Here we will now analyze the obtained results with respect to their coverage.

We will compute the coverage and plot a 2D histogram of the differently gathered treatments. 

This will give us a better idea of the differences - especially between uniform and Latin Hypercube sampling - than simple scatter plots.

First of all, copy the sampling commands which you already implemented into the following cell so we do not have to re-run each cell when changing the sample size.

(And don't forget to scale here as well)

For the Latin Hypercube:
    Make this time use of the 'optimization' keyword and watch the difference

In [10]:
n_samples = 100
np.random.seed(0)

treatments_normal = scale(np.random.normal(0, 1, (n_samples, n_factors)), condition_range)
treatments_uniform = np.random.uniform(condition_range[0], condition_range[1], (n_samples, n_factors))
treatments_lhc = scale(LatinHypercube(d=n_factors, optimization='lloyd').random(n_samples), condition_range)

Let's compute the coverage of the treatments with respect to the design space!

The steps for our analysis look as follows:
1. Compute the target coverage which is $target=\frac{n_\text{samples}}{n_\text{bins}**n_\text{factors}}$
2. Compute 2D Histograms for each of the methods where the number of samples per bin is counted
3. Compute the bin-wide coverage for each method
4. Compute under-, over- and highly over-represented bins ($bin_\text{count}<target$; $bin_\text{count}>target$; $bin_\text{count}>target*threshold$)

In [ ]:
hist_bins = 10
threshold_highly_overrepresented = 2

# compute the target coverage
target_coverage = np.max((n_samples/(hist_bins**n_factors), 1))
threshold_highly_overrepresented = target_coverage * threshold_highly_overrepresented
print(f'Target coverage: {target_coverage} samples per bin')

# compute 2D histograms for each sampling method with the numpy method
hist_normal = np.histogram2d(treatments_normal[:, 0], treatments_normal[:, 1], bins=hist_bins)[0]
hist_uniform = np.histogram2d(treatments_uniform[:, 0], treatments_uniform[:, 1], bins=hist_bins)[0]
hist_lhc = np.histogram2d(treatments_lhc[:, 0], treatments_lhc[:, 1], bins=hist_bins)[0]

# compute the actual coverage for each sampling method
coverage_normal = np.round(np.sum(hist_normal > 0)/(hist_bins**n_factors) * 100, 2)
coverage_uniform = np.round(np.sum(hist_uniform > 0)/(hist_bins**n_factors) * 100, 2)
coverage_lhc = np.round(np.sum(hist_lhc > 0)/(hist_bins**n_factors) * 100, 2)

# compute the amount of underrepresented bins for each sampling method
underrep_normal = round(np.sum(hist_normal < target_coverage)/(hist_bins**n_factors) * 100, 2)
underrep_uniform = round(np.sum(hist_uniform < target_coverage)/(hist_bins**n_factors) * 100, 2)
underrep_lhc = round(np.sum(hist_lhc < target_coverage)/(hist_bins**n_factors) * 100, 2)

# compute the amount of overrepresented bins for each sampling method
overrep_normal = round(np.sum(hist_normal > target_coverage)/(hist_bins**n_factors) * 100, 2)
overrep_uniform = round(np.sum(hist_uniform > target_coverage)/(hist_bins**n_factors) * 100, 2)
overrep_lhc = round(np.sum(hist_lhc > target_coverage)/(hist_bins**n_factors) * 100, 2)

# compute the amount of highly overrepresented bins for each sampling method
highly_overrep_normal = round(np.sum(hist_normal > threshold_highly_overrepresented)/(hist_bins**n_factors) * 100, 2)
highly_overrep_uniform = round(np.sum(hist_uniform > threshold_highly_overrepresented)/(hist_bins**n_factors) * 100, 2)
highly_overrep_lhc = round(np.sum(hist_lhc > threshold_highly_overrepresented)/(hist_bins**n_factors) * 100, 2)

# print the results in a table
print(f'{"-"*25:<25} {"-"*15:<15} {"-"*25:<25} {"-"*25:<25} {"-"*25:<25}')
print(f'{"Sampling method":<25} {"Coverage [%]":<15} {"Underrepresented [%]":<25} {"Overrepresented [%]":<25} {"Highly overrepresented [%]":<25}')
print(f'{"-"*25:<25} {"-"*15:<15} {"-"*25:<25} {"-"*25:<25} {"-"*25:<25}')
print(f'{"Normal distribution":<25} {coverage_normal:<15} {underrep_normal:<25} {overrep_normal:<25} {highly_overrep_normal:<25}')
print(f'{"Uniform distribution":<25} {coverage_uniform:<15} {underrep_uniform:<25} {overrep_uniform:<25} {highly_overrep_uniform:<25}')
print(f'{"Latin Hypercube":<25} {coverage_lhc:<15} {underrep_lhc:<25} {overrep_lhc:<25} {highly_overrep_lhc:<25}')
print(f'{"-"*25:<25} {"-"*15:<15} {"-"*25:<25} {"-"*25:<25} {"-"*25:<25}')


Let's make the corresponding plots!

The plotting is already implemented to make your life a little bit easier. 

You are still very welcome to go through the plotting code on your own as oftentimes this can be a quite challenging part!

In [ ]:
scatter = True

# make a figure with three subplots for the three sampling methods
fig, axes = plt.subplots(1, 3, figsize=(12, 6))

# plot the samples from the normal distribution on the first subplot
axes[0].hist2d(treatments_normal[:, 0], treatments_normal[:, 1], bins=hist_bins, cmap='Blues')
if scatter:
    axes[0].scatter(treatments_normal[:, 0], treatments_normal[:, 1], color='orange')
axes[0].set_title('normal distribution')
axes[0].set_xlabel('Factor 1')
axes[0].set_ylabel('Factor 2')

# plot the samples from the uniform distribution on the second subplot
axes[1].hist2d(treatments_uniform[:, 0], treatments_uniform[:, 1], bins=hist_bins, cmap='Blues')
if scatter:
    axes[1].scatter(treatments_uniform[:, 0], treatments_uniform[:, 1], color='orange')
axes[1].set_title('uniform distribution')
axes[1].set_xlabel('Factor 1')

# plot the LHS samples on the third subplot
axes[2].hist2d(treatments_lhc[:, 0], treatments_lhc[:, 1], bins=hist_bins, cmap='Blues')
if scatter:
    axes[2].scatter(treatments_lhc[:, 0], treatments_lhc[:, 1], color='orange')
axes[2].set_title('Latin Hypercube')
axes[2].set_xlabel('Factor 1')

# set figure title
fig.suptitle('Sampling from')

# share y axis
axes[1].sharey(axes[0])
axes[2].sharey(axes[0])
# hide y axis for the second and third subplot
axes[1].get_yaxis().set_visible(False)
axes[2].get_yaxis().set_visible(False)

plt.show()

After doing all these analysis: What's your impression of the different random sampling methods?

Can you think of use-cases for each of the method?

Can you tell now a difference between the uniform and Latin Hypercube sampling?

# Let's set up our own experiment with these sampling methods!

Let's check if we get significantly different results for the presented random sampling methods with respect to noise, dataset size and dataset structure (i.e. number of repetitions per condition vs number of conditions).
First of all, we have to import the existing classes for the experimental unit, the dataset generator, and the feed-forward-network regressor. You should be familiar with these from the last tutorial.

## Library, package and class imports

In [13]:
import os, sys
import torch
from skorch import NeuralNetRegressor
from sklearn.metrics import mean_squared_error

# Set the path to the project folder
target_folder = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust path as needed
if target_folder not in sys.path:
    sys.path.append(target_folder)

from resources.synthetic import experimental_unit, normal_ground_truth, noise, generate_dataset
from resources.regressors import FFN

Now we have to set up the sampling strategy and generate the dataset. You can use the respective parts from the first tutorial. Set the number of participants, tested conditions, repitions and the noise level as seen before.

In [14]:
# define the dataset parameters

# number of experimental units
n_units = 100

# number of observations per experimental unit
n_conditions = 1000

# number of repetitions per condition
n_repetitions = 1

# amount of noise which we are going to add to the data
noise_level = 0.

# relative amount of train vs test samples
train_ratio = 0.8

In [15]:
experimental_units = None
train_data = None
test_data = None
parameters = None
conditions = None

# add your code here; don't forget that the final class of your dataset should be torch.Tensor
# you can use the generate_dataset method to create a dataset which you can then split into train and test samples
# e.g.:
# dataset, dataset_flat = generate_dataset(experimental_units, conditions, n_repetitions, shuffle=True)
# train_data = dataset_flat[:int(len(dataset_flat)*train_ratio)]
# test_data = dataset_flat[int(len(dataset_flat)*train_ratio):]

Now we can train the regressor based on the generated dataset 

In [ ]:
model = NeuralNetRegressor(FFN(n_units, 2), max_epochs=100)
model.fit(train_data[:, 0:-1], train_data[:, -1][:, None])

prediction = model.predict(test_data[:, 0:-1])
loss = mean_squared_error(test_data[:, -1][:, None].numpy(), prediction)

print(f"Test loss: {loss:.8f}")

Let's inspect the trained model in more detail

In [ ]:
# plot the recovered response times over the conditions
# define the factor levels
x = np.linspace(0, 1)
y = np.linspace(0, 1)
x_mesh, y_mesh = np.meshgrid(x, y)
sample_size = len(x)

# initiate the z array
z_recovered = np.zeros((sample_size, sample_size))
z_real = np.zeros((sample_size, sample_size))

unit_id = 30

# collect the observations
for i in range(sample_size):
    condition = torch.tensor(np.stack((x_mesh[i], y_mesh[i]), axis=-1))
    unit_id_array = torch.full((condition.shape[0], 1), unit_id)
    X = torch.cat((unit_id_array, condition), axis=-1)
    
    z_recovered[i, :] = model.predict(X).reshape(-1)
    z_real[i, :] = experimental_units[unit_id].step((x_mesh[i], y_mesh[i]), noise=False)
    
# make a surface plot to visualize the ground_truth
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot_surface(x_mesh, y_mesh, z_recovered, cmap=cm.Reds, alpha=0.5)
ax.plot_surface(x_mesh, y_mesh, z_real, cmap=cm.Blues, alpha=0.5)
ax.set_title('Real (blue) vs recovered (red) response times by a neural network over all factor levels')
ax.set_xlabel('Ratio')
ax.set_ylabel('Scatterdness')
ax.set_zlabel('Response time')
plt.show()

## Congratulations!

You've finished the tutorial on random sampling!